In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pretty_midi
import librosa
import librosa.display
import gc
from sklearn.preprocessing import StandardScaler
import warnings
from collections import Counter
from torch.utils.data import Dataset
import torch
from pympler import asizeof
import torch.optim as optim 



import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from sklearn.preprocessing import StandardScaler

from Preprocessing import *
#from ExtractGenre import *
from CNN_ExtractGenre import *
from PolyphonicPreprocessing import *
import Util as Util

import DatasetLoader as DL
import Model as M



In [2]:
GenreMapping = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

In [ ]:
def AddZeros(Lists):
   
   maxValList = [np.max(a) for a in Lists]
   maxVal = np.max(maxValList)

   maxArr = np.arange(1, maxVal+1)

   outputList = []

   for i in Lists:
      maskZeros = np.zeros_like(maxArr)
      mask = np.isin(maxArr, i)
      maskZeros[mask] = maxArr[mask]
      outputList.append(maskZeros)

   return np.array(outputList)


def ToPolyphonicBars(track, TicksPerBeat, Velocity, length=16):
   # Since these tracks are all 4/4
   TicksPerBar = TicksPerBeat * 4
   TicksPerSixteenth = TicksPerBar // length
   
   currTime = 0
   ActiveNotes = {}  
   Note = []
   
   for msg in track:
      currTime += msg.time
      
      if msg.type == 'note_on' and msg.velocity > 0:
         # Note starts
         ActiveNotes[msg.note] = (currTime, msg.velocity)
         
      elif (msg.type == 'note_off') or (msg.type == 'note_on' and msg.velocity == 0):
         # Note ends
         if msg.note in ActiveNotes:
            start_time, velocity = ActiveNotes[msg.note]
            end_time = currTime
            
            # Compute position
            start_bar = start_time // TicksPerBar
            end_bar = end_time // TicksPerBar
            start_pos = (start_time % TicksPerBar) // TicksPerSixteenth
            end_pos = (end_time % TicksPerBar) // TicksPerSixteenth
            
            if start_bar == end_bar:
               # Note within single bar
               if start_pos < length and end_pos <= length:
                  Note.append((start_bar, msg.note, start_pos, min(end_pos, length-1), velocity))
            else:
               if start_pos < length:
                  Note.append((start_bar, msg.note, start_pos, length-1, velocity))
               
               for bar in range(start_bar + 1, end_bar):
                  Note.append((bar, msg.note, 0, length-1, velocity))
               
               if end_pos > 0 and end_pos <= length:
                  Note.append((end_bar, msg.note, 0, end_pos-1, velocity))
            
            del ActiveNotes[msg.note]
   
   # Handle any notes that were never turned off
   for note, (start_time, velocity) in ActiveNotes.items():
      start_bar = start_time // TicksPerBar
      start_pos = (start_time % TicksPerBar) // TicksPerSixteenth
      if start_pos < length:
         Note.append((start_bar, note, start_pos, start_pos, velocity))
   
   Bars = {}
   barNumRecord = []
   for barNum, note, start_pos, end_pos, vel in Note:
      if barNum not in Bars:
         Bars[barNum] = np.zeros((128, length), dtype=int)
         barNumRecord.append(barNum+1)


      # Fill the matrix 
      for pos in range(start_pos, end_pos + 1):
         if pos < length:
            if Velocity:
               Bars[barNum][note, pos] = vel
            else:
               Bars[barNum][note, pos] = 1
   
   barList = []
   for barNum, matrix in Bars.items():
      barList.append(matrix)
   
   return barNumRecord, barList


#From all the track (maximum 4) of a given song build the (4x128x16) tensor 
def ToPolyphonicGeneralInfo(mid, Dataset, file, dir, Velocity, HowManyInstruments = 4 ):

   Func_Tempo = lambda t: 60_000_000 / t
   TicksPerBeat = mid.ticks_per_beat
   TrackName = f'{dir}/{file[:-4]}'

   #defining the tempo of file (one for each)
   if len(mid.tracks) > 0:
      InitialTrack = mid.tracks[0]
      for msg in InitialTrack:
         if msg.type == 'set_tempo':

            Tempo = Func_Tempo(msg.tempo)
            break
         else:
            Tempo = 120

   ChannelList = []
   BarsRecordList, BarsList, ProgramList = [], [], []

   #Keeps track of the four family of instrument 
   PianoInstr, GuitarInstr, BassInstr = 0, 0, 0
   PercussInstr, StringInstr, BrassInstr = 0, 0, 0

   for track in mid.tracks:
      #Consider only the tracks that have an instrument in it (remove grabage!!)
      HasProgramChange = any(msg.type == 'program_change' for msg in track)
      Channel = [msg.channel for msg in track if msg.type == 'control_change']

      if len(Channel) == 0:
         continue
      else:
         Channel = Channel[0]
      
      if HasProgramChange or Channel == 9:

         Program = [msg.program for msg in track if msg.type == 'program_change'][0] if Channel != 9 else 150

         #Program = 0 is invalid
         # if Program == 0 and Channel != 9:
         #    continue

         #Conventional channel for percussion instruments
         if Channel == 9 and PercussInstr < 1:
            #Define a new program that identify the percussion inst.
            Program = 150

            BarsRecord, Bars = ToPolyphonicBars(track, TicksPerBeat, Velocity)
            if Bars is None or len(Bars) <4:
               continue
            PercussInstr += 1
            BarsRecordList.append(BarsRecord)
            BarsList.append(Bars)
            ChannelList.append(Channel)
            ProgramList.append(Program)

         #If the instrument is not in the Percussive family
         if Channel != 9:
            #If the instrument is in the family of Piano and ensure only 1 instrument is kept if there are more in the song
            if 1 <= Program <= 8 and PianoInstr < 1:
               #Compute the (128x16) bars matrix for each track
               BarsRecord, Bars = ToPolyphonicBars(track, TicksPerBeat, Velocity)
               if Bars is None or len(Bars) < 4:
                  continue
               PianoInstr += 1
               BarsRecordList.append(BarsRecord)
               BarsList.append(Bars)
               ChannelList.append(Channel)
               ProgramList.append(Program)

            #If the instrument is in the family of Guitar
            elif 25 <= Program <= 32 and GuitarInstr < 1:
               #Compute the (128x16) bars matrix for each track
               BarsRecord, Bars = ToPolyphonicBars(track, TicksPerBeat, Velocity)
               if Bars is None or len(Bars) <4:
                  continue
               GuitarInstr += 1
               BarsRecordList.append(BarsRecord)
               BarsList.append(Bars)
               ChannelList.append(Channel)
               ProgramList.append(Program)

            #If the instrument is in the family of Bass
            elif 33 <= Program <= 40 and BassInstr < 1:
               #Compute the (128x16) bars matrix for each track
               BarsRecord, Bars = ToPolyphonicBars(track, TicksPerBeat, Velocity)
               if Bars is None or len(Bars) <4:
                  continue
               BassInstr += 1
               BarsRecordList.append(BarsRecord)
               BarsList.append(Bars)
               ChannelList.append(Channel)
               ProgramList.append(Program)

            elif 41 <= Program <= 48 and StringInstr < 1:
               #Compute the (128x16) bars matrix for each track
               BarsRecord, Bars = ToPolyphonicBars(track, TicksPerBeat, Velocity)
               if Bars is None or len(Bars) <4:
                  continue
               StringInstr += 1
               BarsRecordList.append(BarsRecord)
               BarsList.append(Bars)
               ChannelList.append(Channel)
               ProgramList.append(Program)


            elif 57 <= Program <= 64 and BrassInstr < 1:
               #Compute the (128x16) bars matrix for each track
               BarsRecord, Bars = ToPolyphonicBars(track, TicksPerBeat, Velocity)
               if Bars is None or len(Bars) <4:
                  continue
               BrassInstr += 1
               BarsRecordList.append(BarsRecord)
               BarsList.append(Bars)
               ChannelList.append(Channel)
               ProgramList.append(Program)

            else:
               continue

      #Only these four instruments
      TotInstr = PianoInstr + GuitarInstr + BassInstr + PercussInstr + StringInstr + BrassInstr
      if  TotInstr == 4:
         break

   if len(ChannelList) != 4:
      global Empty
      Empty += 1
      return Dataset
   
   if len(BarsRecordList) == 0:
      return Dataset

   #Complete array for the number of bars
   FullBarRecord = AddZeros(BarsRecordList)

   #Check the active instrument at bar i:
   FullActiveBars = []
   for i in range(np.shape(FullBarRecord)[1]):
      ActiveBars = np.zeros(4, dtype=int)
      for trackBarsNum in range(np.shape(FullBarRecord)[0]):

         if FullBarRecord[trackBarsNum, i] == 0:
            ActiveBars[trackBarsNum] = 0
         else:
            ActiveBars[trackBarsNum] = 1

      FullActiveBars.append(ActiveBars)
   

   #Taking songs that have at least 4 instruments playing
   # if len(FullBarRecord) < HowManyInstruments:
   #    return Dataset
   
   PolyphonicDataset = []
            #loop through all the bars (active or inactive)
   for i in range(np.shape(FullBarRecord)[1]):

      PolyphonicBars = np.zeros((HowManyInstruments, 128, 16), dtype=int)
                                 #Has shape 4
      for trackBarsNum in range(np.shape(FullBarRecord)[0]):
         
         if FullBarRecord[trackBarsNum, i] == 0:
            EmptyBar = np.zeros((128, 16), dtype = int)
            PolyphonicBars[trackBarsNum, :, :] = EmptyBar

         else:
            FindBar = np.where(BarsRecordList[trackBarsNum] == FullBarRecord[trackBarsNum, i])[0][0]
            PolyphonicBars[trackBarsNum, :, :] = BarsList[trackBarsNum][FindBar]

      
      PolyphonicBars = torch.tensor(PolyphonicBars).to_sparse()

      PolyphonicDataset.append(PolyphonicBars)
         

   #Counts the number of pair of bars
   Dim = len(PolyphonicDataset)//2

   numPair = [(i, i+1) for i in range(2, Dim - 3, 2)]
   BarsPair = [(PolyphonicDataset[i], PolyphonicDataset[i+1]) for i in range(2, Dim - 3, 2)]
   ActiveProgram = [(FullActiveBars[i], FullActiveBars[i+1]) for i in range(2, Dim - 3, 2)]
   FullProgramList = [(ProgramList, ProgramList) for _ in range(2, Dim - 3, 2)]


   #If there is not the track in the dataset, add it
   if TrackName not in Dataset:               
      Dataset[TrackName] = {
         'SongName': [],
         'Bars': [],
         'Program': [],
         'ActiveProgram': [],
         'numBar': [],
         'Tempo': [], 
         'Channel': []
      }

   #Maps the program into one instrument of the same category
   
   #and add the information to the Dataset dictionary
   Dataset[TrackName]['SongName'].extend([(f'{TrackName}', f'{TrackName}') for _ in range(2, Dim - 3, 2)])
   Dataset[TrackName]['Bars'].extend(BarsPair)
   Dataset[TrackName]['Program'].extend(FullProgramList)
   Dataset[TrackName]['ActiveProgram'].extend(ActiveProgram)
   Dataset[TrackName]['numBar'].extend(numPair)
   Dataset[TrackName]['Tempo'].extend([int(Tempo) for _ in range(2, Dim - 3, 2)])
   Dataset[TrackName]['Channel'].extend(ChannelList)

   return Dataset

In [4]:
Path = os.path.realpath('clean_midi/Third Eye Blind/Jumper.1.mid')
mid = mido.MidiFile(Path)

Dataset = {}
file = 'Never Let You Go.mid'
dir = 'Jumper.1'
#test = ToPolyphonicGeneralInfo(mid, Dataset, file, dir, Velocity= 100)

In [5]:
Empty = 0
#After having created the dataset, cathegorize the songs in the corresponding genre
def PolyphonicPreProcessing(nDir = 300, Velocity=False):

   Dataset = {}
   iter = 0

   InputPath = os.path.relpath('clean_midi')

   #Selecting a random number of directory
   all_dirs = [d for d in os.listdir(InputPath) if os.path.isdir(os.path.join(InputPath, d))]

   random_dirs = np.random.choice(all_dirs, nDir)

   for dir in tqdm(random_dirs, desc='Preprocessing'):
      DirPath = os.path.join(InputPath, dir)

      if not os.path.isdir(DirPath):
         continue

      #Real all the file in each folder
      for file in os.listdir(DirPath):
         iter += 1

         FilePath = os.path.join(DirPath, file)

         #Cleaned monophonic: Some songs are corrupted:
         mid = Func_CorruptedFile(FilePath, file, dir)

         if mid is None:
            continue

         Dataset = ToPolyphonicGeneralInfo(mid, Dataset, file, dir, Velocity)

   print(iter, Empty)

   return Dataset

   #Load the file that maps each song in the corresponding genre
   with open('GenreDataset.pkl', 'rb') as f:
      GenreDataset = pickle.load(f)

   GenreMapping = {0: 'metal', 1: 'disco', 2: 'classical', 3: 'hiphop', 4: 'jazz',
          5: 'country', 6: 'pop', 7: 'blues', 8: 'raggae', 9: 'rock'}

   MappedDataset = {}
   for key in Dataset.keys():
      
      try:
         GenreDataset[key]
      except:
         continue

               #Extrapolate genre from the dataset
      Genre = GenreDataset[key][0]
      value = Dataset[key]

      if GenreMapping[Genre] not in MappedDataset:
         MappedDataset[GenreMapping[Genre]] = {
            'SongName': [],
            'Bars': [],
            'Program': [],
            'ActiveProgram': [],
            'numBar': [],
            'Tempo': [], 
            'Channel': []
         }

      #remaps the dataset into the one cathegorized by genre
      MappedDataset[GenreMapping[Genre]]['SongName'].extend(value['SongName'])
      MappedDataset[GenreMapping[Genre]]['Bars'].extend(value['Bars'])
      MappedDataset[GenreMapping[Genre]]['Program'].extend(value['Program'])
      MappedDataset[GenreMapping[Genre]]['ActiveProgram'].extend(value['ActiveProgram'])
      MappedDataset[GenreMapping[Genre]]['numBar'].extend(value['numBar'])
      MappedDataset[GenreMapping[Genre]]['Tempo'].extend(value['Tempo'])
      MappedDataset[GenreMapping[Genre]]['Channel'].extend(value['Channel'])



   FinalDict = {}
   for key in MappedDataset.keys():
      SN = MappedDataset[key]['SongName']
      Bars = MappedDataset[key]['Bars']
      Prog = MappedDataset[key]['Program']
      AP = MappedDataset[key]['ActiveProgram']
      nB = MappedDataset[key]['numBar']
      T = MappedDataset[key]['Tempo']
      Ch = MappedDataset[key]['Channel']


      list = []
      for i in range(len(SN)):
         dict = {
            'SongName': SN[i],
            'Bars': Bars[i],
            'Program': Prog[i],
            'ActiveProgram': AP[i],
            'numBar': nB[i],
            'Tempo': T[i],
            'Channel': Ch[i]
         }
         list.append(dict)

      FinalDict[key] = list

   for key in FinalDict.keys():
      for i in reversed(range(len(FinalDict[key]))):
         if torch.sum(FinalDict[key][i]['Bars'][0]) == 0 or torch.sum(FinalDict[key][i]['Bars'][1]) == 0:
            del FinalDict[key][i]

   return FinalDict


In [6]:
PolyphonicDataset = PolyphonicPreProcessing(nDir = 10)


Preprocessing: 100%|██████████| 10/10 [00:02<00:00,  4.97it/s]

49 36


In [ ]:
PolyphonicDataset['Black/Wonderful Life.1']['Channel']